<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M2_mapping_european_ai-class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('https://github.com/aaubs/ds-master/raw/main/data/ai_landscape_links.csv.gz', '\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
len(data)

11443

In [38]:
data.dl_slot.nunique()

721

In [6]:
edges = []

for i in data.iterrows():
  targets = i[1]['links'].split(',')
  edges_i = [(i[1]['dl_slot'], j) for j in targets]
  edges.extend(edges_i)

In [7]:
edges[:10]

[('ebenbuild.com', 'umm.de'),
 ('ebenbuild.com', 'nvidia.com'),
 ('ebenbuild.com', 'diepresse.com'),
 ('ebenbuild.com', 'twitter.com'),
 ('ebenbuild.com', 'unternehmertum.de'),
 ('ebenbuild.com', 'lnkd.in'),
 ('ebenbuild.com', 'materialise.com'),
 ('ebenbuild.com', 'munich-startup.de'),
 ('ebenbuild.com', 'uni-muenchen.de'),
 ('ebenbuild.com', 'gepflegt-durchatmen.de')]

In [13]:
import networkx as nx
from community import community_louvain

In [10]:
G = nx.DiGraph()

In [11]:
G.add_edges_from(edges)

In [14]:
centrality_dgr = nx.degree_centrality(G)
centrality_in = nx.in_degree_centrality(G)
centrality_out = nx.out_degree_centrality(G)
centrality_eig = nx.eigenvector_centrality_numpy(G)

In [15]:
nx.set_node_attributes(G, centrality_dgr, 'dgr')
nx.set_node_attributes(G, centrality_eig, 'eig')
nx.set_node_attributes(G, centrality_in, 'in')
nx.set_node_attributes(G, centrality_out, 'out')

In [16]:
# This is how you turn a Graph object (NetworkX) to a Dataframe
nodes_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [24]:
nodes_df.sort_values(['eig'], ascending=False)

,dgr,eig,in,out
facebook.com,0.055645,3.727510e-01,0.055645,0.000000
twitter.com,0.071129,3.727510e-01,0.071129,0.000000
youtube.com,0.043387,2.511509e-01,0.043387,0.000000
linkedin.com,0.104516,2.363147e-01,0.102581,0.001935
google.com,0.042903,1.918059e-01,0.042903,0.000000
...,...,...,...,...
shopify.com,0.001452,-4.455934e-17,0.001452,0.000000
privacyshield.gov,0.005161,-4.714980e-17,0.005161,0.000000
aistartuplandscape.se,0.000645,-4.944275e-17,0.000645,0.000000
kke.co.jp,0.000161,-4.958109e-17,0.000161,0.000000


In [43]:
to_kick_ids = nodes_df[((nodes_df.dgr > nodes_df.dgr.quantile(0.90)) & (nodes_df.out == 0))]

In [46]:
to_kick_ids

,dgr,eig,in,out
nvidia.com,0.005484,8.901744e-02,0.005484,0.0
twitter.com,0.071129,3.727510e-01,0.071129,0.0
appliedai.de,0.002903,-2.569605e-17,0.002903,0.0
t3n.de,0.001452,4.680985e-18,0.001452,0.0
instagram.com,0.033871,7.020584e-02,0.033871,0.0
...,...,...,...,...
creativecommons.org,0.001774,1.409818e-17,0.001774,0.0
opera.com,0.002419,4.450872e-02,0.002419,0.0
latribune.fr,0.001452,1.024613e-18,0.001452,0.0
vinnova.se,0.001290,-8.164557e-18,0.001290,0.0


In [47]:
nodes_df.drop(to_kick_ids.index, inplace=True)

In [51]:
nodes_df[((nodes_df['in'] > nodes_df['in'].quantile(0.90)) & (nodes_df.out == 0))][:50]

,dgr,eig,in,out
unternehmertum.de,0.001129,-4.036245e-17,0.001129,0.0
lnkd.in,0.001129,4.450872e-02,0.001129,0.0
munich-startup.de,0.000645,1.146457e-17,0.000645,0.0
uni-muenchen.de,0.000484,4.450872e-02,0.000484,0.0
tum.de,0.001129,4.450872e-02,0.001129,0.0
ki-verband.de,0.001129,8.733275e-18,0.001129,0.0
dw.com,0.000484,4.450872e-02,0.000484,0.0
de.digital,0.000645,2.754017e-18,0.000645,0.0
flickr.com,0.000645,1.364363e-01,0.000645,0.0
hbr.org,0.000968,1.449603e-17,0.000968,0.0


In [52]:
# Subset the graph keeping only nodes with degree > 1
G = nx.subgraph(G, nodes_df.index)

In [53]:
len(G.nodes())

6088

In [55]:
partition = community_louvain.best_partition(G.to_undirected()) #that will take some time...

In [58]:
nx.set_node_attributes(G, partition, 'partition')

In [59]:
# This is how you turn a Graph object (NetworkX) to a Dataframe
nodes_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [69]:
nodes_df.partition.value_counts()[50:70]

64    37
62    31
60    30
82    27
35    24
86    17
81    14
65    14
89    13
69    12
84    11
68    10
43     9
79     7
30     7
61     5
90     4
27     3
71     2
70     2
Name: partition, dtype: int64

In [72]:
nodes_df[nodes_df.partition == 62]

,dgr,eig,in,out,partition
seapattern.com,0.000645,2.037151e-18,0.000000,0.000645,62
lead.se,0.000323,8.869044e-18,0.000323,0.000000,62
populariswp.com,0.000161,1.987429e-18,0.000161,0.000000,62
hubspotusercontent30.net,0.000484,-2.325091e-18,0.000484,0.000000,62
unitedrobots.ai,0.001774,2.245351e-18,0.000000,0.001774,62
imy.se,0.000484,5.926866e-18,0.000484,0.000000,62
unitedrobots.se,0.000161,7.916755e-18,0.000161,0.000000,62
kw.be,0.000161,7.300306e-18,0.000161,0.000000,62
podbean.com,0.000161,7.089844e-18,0.000161,0.000000,62
untiedrobots.se,0.000161,9.326057e-18,0.000161,0.000000,62


In [75]:
nx.write_gexf(G,'G.gexf')